<a href="https://colab.research.google.com/github/davicn/poc_vqa/blob/main/evaluating_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependências

In [ ]:
!pip install python-Levenshtein pyxDamerauLevenshtein datasets transformers Pillow uform --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
from PIL import Image
from matplotlib.pyplot import imshow
from datasets import load_dataset
from transformers import (BlipProcessor, BlipForQuestionAnswering, Pix2StructProcessor, Pix2StructForConditionalGeneration)
from Levenshtein import distance as lev
from pyxdameraulevenshtein  import damerau_levenshtein_distance as dalev
from uform.gen_model import VLMForCausalLM, VLMProcessor



## Base para avaliação

In [ ]:
## Base para avaliação

df1 = pd.read_csv('/content/drive/MyDrive/hvar/validate_data_1000.csv')
dataset = load_dataset("vikhyatk/lnqa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df1.columns = ['image_id', 'subset', 'original_url', 'original_landing_url', 'license', 'author_profile_url', 'author', 'title', 'original_size', 'original_md5', 'thumbnail300k_url', 'rotation']

In [ ]:
df1 = df1.astype({'image_id':str})

df1.head()

,image_id,subset,original_url,original_landing_url,license,author_profile_url,author,title,original_size,original_md5,thumbnail300k_url,rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,d05c3e451f79174d,train,https://c6.staticflickr.com/8/7375/13791738583...,https://www.flickr.com/photos/jeepersmedia/137...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/jeepersmedia/,Mike Mozart,Pez Kids Breakfast Cereal Prizes,972558,KoVMrVWKftzadg1EGr6nCg==,https://c1.staticflickr.com/8/7375/13791738583...,0.0
2,5bde9ad21e82deed,train,https://c6.staticflickr.com/1/223/495658260_89...,https://www.flickr.com/photos/8106459@N07/4956...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/8106459@N07/,David~O,Isabella Lake,234199,dsfj7rNAbGq67OzPRAGdsQ==,https://c2.staticflickr.com/1/223/495658260_85...,0.0
3,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
4,3231e75bd7120b7a,train,https://c3.staticflickr.com/4/3654/3619297168_...,https://www.flickr.com/photos/cckaiser/3619297168,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/cckaiser/,Colin,DSC_0096,2570210,195AcCHMqin3MYTv+pTDuA==,https://c4.staticflickr.com/4/3654/3619297168_...,0.0


In [ ]:
ids = df1['image_id'].to_list()

l = []

for i in dataset['train']:
    if i['image_id'] in ids:
        l.append(i)

In [ ]:
df2 = pd.DataFrame(l)
df2 = df2.astype({'image_id':str})


In [ ]:
df2.head()

,image_id,question,answer
0,600d95f478094765,What type of structures can be seen in the image?,There are houses in the image.
1,600d95f478094765,What type of vegetation is present in the image?,"There are trees, plants, and grass in the image."
2,600d95f478094765,What architectural features can be seen in the...,"There are windows, doors, pillars, and lamps i..."
3,600d95f478094765,What is the path used for in the image?,The path is likely used for walking or traveli...
4,600d95f478094765,What is the board used for in the image?,The purpose of the board in the image is not c...


In [ ]:
df = pd.merge(df1, df2, on='image_id', how='inner')

## Métricas

## BLIP: Bootstrapping Language-Image Pre-training for Unified Vision-Language Understanding and Generation

- Model card for BLIP trained on visual question answering- base architecture (with ViT base backbone).
- https://huggingface.co/Salesforce/blip-vqa-base


In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

In [ ]:

def get_img(url:str) -> any:
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))

    return img


def inference(img:any, question: str) -> any:
    inputs = processor(img, question, return_tensors="pt")

    out = model.generate(**inputs)

    return processor.decode(out[0], skip_special_tokens=True)


def main(item:any) -> str:
    img = get_img(url=item['original_url'])
    predicted_answer = inference(img=img, question=item['question'])
    return predicted_answer



In [ ]:
metrics = []

for index, row in df.iterrows():
    try:
        d = {}
        predicted_answer = main(row)
        acc = lev(row['answer'], predicted_answer)
        metrics.append(acc)
    except:
        continue


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Model card for MatCha - fine-tuned on ChartQA

- This model is the MatCha model, fine-tuned on Chart2text-pew dataset.
- https://huggingface.co/google/matcha-chartqa

In [ ]:
processor = Pix2StructProcessor.from_pretrained('google/matcha-chartqa')
model = Pix2StructForConditionalGeneration.from_pretrained('google/matcha-chartqa')

metrics2 = []

for index, row in df.iterrows():
    try:
        d = {}
        img = get_img(url=row['original_url'])
        inputs = processor(texts=[row['question']], images=[img], return_tensors="pt")

        with torch.inference_mode():
            output = model.generate(
                **inputs,
                do_sample=False,
                use_cache=True,
                max_new_tokens=128,
                eos_token_id=32001,
                pad_token_id=processor.tokenizer.pad_token_id
                )
        prompt_len = inputs["input_ids"].shape[1]
        predicted_answer = processor.batch_decode(output[:, prompt_len:])[0]
        acc = lev(row['answer'], predicted_answer)
        metrics2.append(acc)
    except:
        continue

Arial.TTF:   0%|          | 0.00/276k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
model = VLMForCausalLM.from_pretrained("unum-cloud/uform-gen-chat")
processor = VLMProcessor.from_pretrained("unum-cloud/uform-gen-chat")

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/974M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/71.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
metrics3 = []

for index, row in df.iloc[:2].iterrows():
    try:
        d = {}
        img = get_img(url=row['original_url'])
        inputs = processor(texts=[row['question']], images=[img], return_tensors="pt")

        with torch.inference_mode():
            output = model.generate(
                **inputs,
                do_sample=False,
                use_cache=True,
                max_new_tokens=128,
                eos_token_id=32001,
                pad_token_id=processor.tokenizer.pad_token_id
                )
        prompt_len = inputs["input_ids"].shape[1]
        predicted_answer = processor.batch_decode(output[:, prompt_len:])[0]
        acc = lev(row['answer'], predicted_answer)
        metrics3.append(acc)
    except:
        continue

In [ ]:
import numpy as np

In [ ]:
print(np.min(metrics))
print(np.mean(metrics))
print(np.var(metrics))

13
50.0
701.6690647482014


In [ ]:
print(np.min(metrics2))
print(np.mean(metrics2))
print(np.var(metrics2))

12
52.62589928057554
719.4427824646757
